In [1]:
import zipfile
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

--2023-10-22 03:39:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   216MB/s    in 8.9s    

2023-10-22 03:39:45 (163 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [12]:
!wget -O Sherlock.txt https://www.gutenberg.org/ebooks/48320.txt.utf-8

--2023-10-22 03:42:49--  https://www.gutenberg.org/ebooks/48320.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-22 03:42:49--  http://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-22 03:42:49--  https://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622314 (608K) [text/plain]
Saving to: ‘Sherlock.txt’

Sherlock.txt        100%[===================>] 607.73

In [2]:
with open("Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()

In [3]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|no|No)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = ['<SOS> ' + s.strip() + ' <EOS>' for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [4]:
book = split_into_sentences(book)

In [5]:
book[1000:1010]

['<SOS> I am going through the City first, and we can have some lunch on the way. <EOS>',
 '<SOS> I observe that there is a good deal of German music on the programme, which is rather more to my taste than Italian or French. <EOS>',
 '<SOS> It is introspective, and I want to introspect. <EOS>',
 '<SOS> Come along! <EOS>',
 '<SOS> ”  We travelled by the Underground as far as Aldersgate; and a short walk took us to Saxe-Coburg Square, the scene of the singular story which we had listened to in the morning. <EOS>',
 '<SOS> It was a poky, little, shabby-genteel place, where four lines of dingy two-storied brick houses looked out into a small railed-in enclosure, where a lawn of weedy grass and a few clumps of faded laurel bushes made a hard fight against a smoke-laden and uncongenial atmosphere. <EOS>',
 '<SOS> Three gilt balls and a brown board with “JABEZ WILSON” in white letters, upon a corner house, announced the place where our red-headed client carried on his business. <EOS>',
 '<SOS

In [6]:
maxTokens = 600
token = Tokenizer(num_words=maxTokens,
                  filters='!"“”#$%&()*+,.-/:;=?@[\\]^_`{|}~\t\n\ufeff\u2002', lower=True,
                  split=' ', char_level=False, oov_token="UNK")
token.fit_on_texts(book)

In [7]:
sequences = token.texts_to_sequences(book)

In [8]:
textSequences = token.sequences_to_texts(sequences)

In [9]:
maxLen = len(max(sequences, key=len))

In [10]:
def create_dataset(sequence):
    inputs = []
    outputs = []
    for i, sent in enumerate(sequence):
        for j in range(1, len(sent)-1):
          inputs.append(sent[:j])
          outputs.append([sent[j]])

    return inputs, outputs

In [11]:
X, y = create_dataset(sequences)

In [12]:
token.sequences_to_texts([X[15]])

['<sos> the project gutenberg UNK of the UNK of sherlock holmes this UNK is for the']

In [13]:
maxLen = len(max(X, key=len))
train_sequences = pad_sequences(X, maxlen=maxLen)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])
num_words=len(dictionary)+1
train_sequences_y = np.array(tf.keras.utils.to_categorical(y, num_classes=num_words))

In [14]:
import codecs
import IPython.display as ipd
from IPython.display import Audio, update_display
from ipywidgets import IntProgress

#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('./crawl-300d-2M.vec', encoding='utf-8')
N = 2000000
bar = IntProgress(min=0, max=N)
ipd.display(bar)

i = 0
for n, line in enumerate(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if n//(N//100) > i:
          bar.value = n
          i += 1
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


IntProgress(value=0, max=2000000)

found 1999996 word vectors


In [15]:
embed_dim=300
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(train_sequences, train_sequences_y, test_size=0.3)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization, LSTM
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

def softMaxOverTime(x):
    return softmax(x, axis=0)

In [45]:
nb_words = num_words #Vocabulary size

model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxLen, trainable=True))
model.add(LSTM(50, return_sequences=True, activation="tanh"))
model.add(Dense(100, activation='relu'))
model.add(Dense(nb_words, activation='softmax'))

In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 101, 300)          2581200   
                                                                 
 lstm_3 (LSTM)               (None, 101, 50)           70200     
                                                                 
 dense_6 (Dense)             (None, 101, 100)          5100      
                                                                 
 dense_7 (Dense)             (None, 101, 8604)         869004    
                                                                 
Total params: 3,525,504
Trainable params: 3,525,504
Non-trainable params: 0
_________________________________________________________________


In [38]:
opt = optimizers.Adam(learning_rate=0.01)
callbackROP = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [42]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=256, epochs=50, callbacks=[callbackROP])

Epoch 1/50


ValueError: in user code:

    File "c:\Users\Gonzalo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Gonzalo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Gonzalo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Gonzalo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Gonzalo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Us

In [161]:
toNum = token.texts_to_sequences(["<SOS> am"])

In [162]:
aver = token.sequences_to_texts([[1]])
aver

['UNK']

In [163]:
toNum = pad_sequences(toNum, maxlen=maxLen, padding="post")

In [164]:
out = model.predict(toNum)

1/1 [==============================] - 0s 378ms/step


In [165]:
np.argmax(out)

1